In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

In [ ]:
# File Paths
dataset_path = "/Users/nitasneemliala/Dataset/Train"

In [3]:
# Image size and batch settings
img_size = 64
batch_size = 32

In [4]:
# Data Augmentation and Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2  # 80% train, 20% test
)

In [5]:
# Training Data
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Testing Data
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Keep order for evaluation
)

Found 132542 images belonging to 28 classes.
Found 33128 images belonging to 28 classes.


In [6]:
# Check correct label mapping
print("Class Labels:", train_generator.class_indices)

Class Labels: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'Nothing': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'Space': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27}


In [10]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Output size matches the number of ASL letters
])

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, validation_data=test_generator, epochs=5)

Epoch 1/5


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4142/4142 ━━━━━━━━━━━━━━━━━━━━ 289s 69ms/step - accuracy: 0.7949 - loss: 0.6693 - val_accuracy: 0.9861 - val_loss: 0.0509
Epoch 2/5
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 284s 68ms/step - accuracy: 0.9903 - loss: 0.0296 - val_accuracy: 0.9690 - val_loss: 0.1467
Epoch 3/5
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 291s 70ms/step - accuracy: 0.9950 - loss: 0.0164 - val_accuracy: 0.9749 - val_loss: 0.1265
Epoch 4/5
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 299s 72ms/step - accuracy: 0.9963 - loss: 0.0125 - val_accuracy: 0.9791 - val_loss: 0.1019
Epoch 5/5
4142/4142 ━━━━━━━━━━━━━━━━━━━━ 287s 69ms/step - accuracy: 0.9971 - loss: 0.0092 - val_accuracy: 0.9910 - val_loss: 0.0713


In [12]:
model.save("asl_model.h5")
print("Model Saved")

Model Saved


In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

1036/1036 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step - accuracy: 0.9884 - loss: 0.0492
Test Accuracy: 0.9909744262695312


In [14]:
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

1036/1036 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step


In [ ]:
correct_pred = np.sum(y_true == y_pred)
accuracy = correct_pred / len(y_true)
print(f"Predictions Accuracy: {accuracy}")

Predictions Accuracy: 0.9909744023182806
Predictions Accuracy: 0.9909744023182806
